In [1]:
import os
import math
from pathlib import Path
import numpy as np
import pandas as pd
from osim.env import ProstheticsEnv
from td3 import TD3
from replay_buffer import ReplayBuffer


In [2]:
CONFIG = {
    "env": {
        "integrator_accuracy": 1e-2,
    },
    "model": {
        "architecture": "TD3",
    },
    "training": {
        "start_timesteps": 1e4, # How many time steps purely random policy is run for
        "eval_freq": 5e3, # How often (time steps) we evaluate
        "max_timesteps": 1e6, # Max time steps to train for
        "max_episode_steps": 300, # Max number of steps to run for a single episode
        "expl_noise": 0.1, # Std of Gaussian exploration noise
        "batch_size": 100, # Batch size for both actor and critic
        "discount": 0.99, # Discount factor
        "tau": 0.005, # Target network update rate
        "policy_noise": 0.2, # Noise added to target policy during critic update
        "noise_clip": 0.5, # Range to clip target policy noise
        "policy_freq": 2, # Frequency of delayed policy updates
    }
}


In [3]:
OUTPUT_DIR = Path('.')
LOGS_DIR = OUTPUT_DIR/'logs'
CHECKPOINTS_DIR = OUTPUT_DIR/'checkpoints'
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)
os.makedirs(CHECKPOINTS_DIR, exist_ok=True)


### Observation Hacking

- Rewrite all joint_pos, body_pos to be relative to mass_center_pos
- Subtract mass_center_vel and mass_center_acc from joint_vel, body_vel, joint_acc, body_acc?
- Either compute jounce/snap, or pass multiple timesteps, or just pass acceleration from past 3 timesteps?

Initial Env Observation:
```
{
    'joint_pos': {
        'ground_pelvis': [0.0, 0.0, 0.0, 0.0, 0.94, 0.0],
        'hip_r': [0.0, 0.0, 0.0],
        'knee_r': [0.0],
        'ankle_r': [0.0],
        'hip_l': [0.0, 0.0, 0.0],
        'knee_l': [0.0],
        'ankle_l': [0.0],
        'subtalar_l': [],
        'mtp_l': [],
        'back': [-0.0872665],
        'back_0': []
    },
    'joint_vel': {
        'ground_pelvis': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        'hip_r': [0.0, 0.0, 0.0],
        'knee_r': [0.0],
        'ankle_r': [0.0],
        'hip_l': [0.0, 0.0, 0.0],
        'knee_l': [0.0],
        'ankle_l': [0.0],
        'subtalar_l': [],
        'mtp_l': [],
        'back': [0.0],
        'back_0': []
    },
    'joint_acc': {
        'ground_pelvis': [34.07237489546962, 3.219284560937942, 0.021285761200362296, 13.997154494145377, 0.8655672359505977, -0.6156967622871027],
        'hip_r': [-194.74323476194263, -4.441803696780512, 1.5931700403370996e-14],
        'knee_r': [305.46152469620915],
        'ankle_r': [9636.363025843913],
        'hip_l': [-208.86020665024324, 3.5702556374966354, -4.2521541843143495e-14],
        'knee_l': [399.3192427973721],
        'ankle_l': [809.4478175113452],
        'subtalar_l': [],
        'mtp_l': [],
        'back': [-2.3092638912203256e-14],
        'back_0': []
    },
    'body_pos': {
        'pelvis': [0.0, 0.94, 0.0],
        'femur_r': [-0.0707, 0.8738999999999999, 0.0835],
        'pros_tibia_r': [-0.07519985651753601, 0.47807930355164957, 0.0835],
        'pros_foot_r': [-0.07519985651753601, 0.04807930355164958, 0.0835],
        'femur_l': [-0.0707, 0.8738999999999999, -0.0835],
        'tibia_l': [-0.07519985651753601, 0.47807930355164957, -0.0835],
        'talus_l': [-0.07519985651753601, 0.04807930355164958, -0.0835],
        'calcn_l': [-0.123969856517536, 0.006129303551649576, -0.09142],
        'toes_l': [0.05483014348246398, 0.004129303551649576, -0.0925],
        'torso': [-0.1007, 1.0214999999999999, 0.0],
        'head': [-0.052764320996907754, 1.5694070821576522, 0.0]
    },
    'body_vel': {
        'pelvis': [0.0, 0.0, 0.0],
        'femur_r': [0.0, 0.0, 0.0],
        'pros_tibia_r': [0.0, 0.0, 0.0],
        'pros_foot_r': [0.0, 0.0, 0.0],
        'femur_l': [0.0, 0.0, 0.0],
        'tibia_l': [0.0, 0.0, 0.0],
        'talus_l': [0.0, 0.0, 0.0],
        'calcn_l': [0.0, 0.0, 0.0],
        'toes_l': [0.0, 0.0, 0.0],
        'torso': [0.0, 0.0, 0.0],
        'head': [0.0, 0.0, 0.0]
    },
    'body_acc': {
        'pelvis': [13.997154494145377, 0.8655672359505977, -0.6156967622871027],
        'femur_r': [16.25111583579615, -1.812159929997423, -0.826986568448235],
        'pros_tibia_r': [-49.070641675940735, 0.12065763836075294, -0.34299240980632545],
        'pros_foot_r': [13.18934420084581, 0.12065763836075294, 0.18269081860597952],
        'femur_l': [16.24756111367569, -1.2745394083207864, -0.826986568448235],
        'tibia_l': [-55.19198970892064, 1.093538716356541, -0.6879691696202773],
        'talus_l': [41.356517039396714, 1.093538716356541, -0.537051606700039],
        'calcn_l': [84.73177709400595, -49.336407951145645, -0.5212902634646581],
        'toes_l': [86.79971256249173, 135.5386990655368, -0.5243942154141731],
        'torso': [11.220255940164602, -2.565520916023193, -0.35118159441778396],
        'head': [-7.448239570993795, -0.9322384901609437, 1.4116668685846663]
    },
    'body_pos_rot': {
        'pelvis': [-0.0, 0.0, -0.0],
        'femur_r': [-0.0, 0.0, -0.0],
        'pros_tibia_r': [-0.0, 0.0, -0.0],
        'pros_foot_r': [-0.0, 0.0, -0.0],
        'femur_l': [-0.0, 0.0, -0.0],
        'tibia_l': [-0.0, 0.0, -0.0],
        'talus_l': [-0.0, 0.0, -0.0],
        'calcn_l': [-0.0, 0.0, -0.0],
        'toes_l': [-0.0, 0.0, -0.0],
        'torso': [-0.0, 0.0, -0.0872665],
        'head': [-0.0, 0.0, -0.0872665]
    },
    'body_vel_rot': {
        'pelvis': [0.0, 0.0, 0.0],
        'femur_r': [0.0, 0.0, 0.0],
        'pros_tibia_r': [0.0, 0.0, 0.0],
        'pros_foot_r': [0.0, 0.0, 0.0],
        'femur_l': [0.0, 0.0, 0.0],
        'tibia_l': [0.0, 0.0, 0.0],
        'talus_l': [0.0, 0.0, 0.0],
        'calcn_l': [0.0, 0.0, 0.0],
        'toes_l': [0.0, 0.0, 0.0],
        'torso': [0.0, 0.0, 0.0],
        'head': [0.0, 0.0, 0.0]
    },
    'body_acc_rot': {
        'pelvis': [3.219284560937942, 0.021285761200362296, 34.07237489546962],
        'femur_r': [-1.2225191358425698, 0.021285761200378228, -160.670859866473],
        'pros_tibia_r': [-1.2225191358425698, 0.021285761200378228, 144.79066482973616],
        'pros_foot_r': [-1.2225191358425698, 0.021285761200378228, 9781.15369067365],
        'femur_l': [-0.35097107655869353, 0.021285761200404818, -174.7878317547736],
        'tibia_l': [-0.35097107655869353, 0.021285761200404818, 224.5314110425985],
        'talus_l': [-0.35097107655869353, 0.021285761200404818, 1033.9792285539438],
        'calcn_l': [-0.35097107655869353, 0.021285761200404818, 1033.9792285539438],
        'toes_l': [-0.35097107655869353, 0.021285761200404818, 1033.9792285539438],
        'torso': [3.219284560937942, 0.021285761200362296, 34.0723748954696],
        'head': [3.219284560937942, 0.021285761200362296, 34.0723748954696]
    },
    'forces': {
        'abd_r': [219.6613927253564],
        'add_r': [144.87433100305103],
        'hamstrings_r': [194.30030504346755],
        'bifemsh_r': [42.728811234363775],
        'glut_max_r': [171.7873509605573],
        'iliopsoas_r': [158.01207984383657],
        'rect_fem_r': [99.0329705435046],
        'vasti_r': [436.79388413623326],
        'abd_l': [219.6613927253564],
        'add_l': [144.87433100305103],
        'hamstrings_l': [194.30030504346755],
        'bifemsh_l': [42.728811234363775],
        'glut_max_l': [171.7873509605573],
        'iliopsoas_l': [158.01207984383657],
        'rect_fem_l': [99.0329705435046],
        'vasti_l': [436.79388413623326],
        'gastroc_l': [273.0178325689043],
        'soleus_l': [370.0059951709156],
        'tib_ant_l': [104.05059952034294],
        'ankleSpring': [-0.0],
        'pros_foot_r_0': [-1.3573320551122304e-12, -388.7553514927188, 0.0, 32.46107184964201, -1.1333722660187127e-13, 3.726164264482634, 1.3573320551122304e-12, 388.7553514927188, 0.0, 0.0, 0.0, 25.50818238819416, 1.3573320551122304e-12, 388.7553514927188, 0.0, 0.0, 0.0, 25.50818238819416],
        'foot_l': [-1.7615592933859115e-12, -504.53063397142085, 0.0, -46.45915575255036, 1.622112753284362e-13, -4.943148065393853, 6.786660275561278e-13, 194.37767574636297, 0.0, 0.0, 0.0, 5.831330272390875, 1.0828932658297836e-12, 310.1529582250579, 0.0, 0.0, 0.0, 6.203059164501164, 1.0828932658297836e-12, 310.1529582250579, 0.0, 0.0, 0.0, 6.203059164501164],
        'HipLimit_r': [0.0, 0.0],
        'HipLimit_l': [0.0, 0.0],
        'KneeLimit_r': [-0.0, 0.0],
        'KneeLimit_l': [-0.0, 0.0],
        'AnkleLimit_r': [0.0, 0.0],
        'AnkleLimit_l': [0.0, 0.0],
        'HipAddLimit_r': [0.0, 0.0],
        'HipAddLimit_l': [0.0, 0.0]
    },
    'muscles': {
        'abd_r': {
            'activation': 0.05,
            'fiber_length': 0.07752306863700548,
            'fiber_velocity': 1.1700156898117815e-13,
            'fiber_force': 219.6613927253564
        },
        'add_r': {
            'activation': 0.05,
            'fiber_length': 0.05526137592854144,
            'fiber_velocity': 5.531257930905764e-11,
            'fiber_force': 146.25768888087705
        },
        'hamstrings_r': {
            'activation': 0.05,
            'fiber_length': 0.06355896214015513,
            'fiber_velocity': 2.1056261406660054e-14,
            'fiber_force': 202.45627069225887
        },
        'bifemsh_r': {
            'activation': 0.05,
            'fiber_length': 0.13434264681417835,
            'fiber_velocity': 9.542198984660805e-17,
            'fiber_force': 45.09919197278584
        },
        'glut_max_r': {
            'activation': 0.05,
            'fiber_length': 0.16084824667171801,
            'fiber_velocity': 1.0181982508865008e-12,
            'fiber_force': 171.7873509605573
        },
        'iliopsoas_r': {
            'activation': 0.05,
            'fiber_length': 0.13005768603600326,
            'fiber_velocity': 3.347183497651294e-11,
            'fiber_force': 159.26525950285387
        },
        'rect_fem_r': {
            'activation': 0.05,
            'fiber_length': 0.06027044615978444,
            'fiber_velocity': 2.2362024955832438e-15,
            'fiber_force': 99.63652479982161
        },
        'vasti_r': {
            'activation': 0.05,
            'fiber_length': 0.07890756873654925,
            'fiber_velocity': 6.168233828156989e-15,
            'fiber_force': 437.7385693769557
        },
        'abd_l': {
            'activation': 0.05,
            'fiber_length': 0.07752306863700548,
            'fiber_velocity': 1.1700156898117815e-13,
            'fiber_force': 219.6613927253564
        },
        'add_l': {
            'activation': 0.05,
            'fiber_length': 0.05526137592854144,
            'fiber_velocity': 5.531257930905764e-11,
            'fiber_force': 146.25768888087705
        },
        'hamstrings_l': {
            'activation': 0.05,
            'fiber_length': 0.06355896214015513,
            'fiber_velocity': 2.1056261406660054e-14,
            'fiber_force': 202.45627069225887
        },
        'bifemsh_l': {
            'activation': 0.05,
            'fiber_length': 0.13434264681417835,
            'fiber_velocity': 9.542198984660805e-17,
            'fiber_force': 45.09919197278584
        },
        'glut_max_l': {
            'activation': 0.05,
            'fiber_length': 0.16084824667171801,
            'fiber_velocity': 1.0181982508865008e-12,
            'fiber_force': 171.7873509605573
        },
        'iliopsoas_l': {
            'activation': 0.05,
            'fiber_length': 0.13005768603600326,
            'fiber_velocity': 3.347183497651294e-11,
            'fiber_force': 159.26525950285387
        },
        'rect_fem_l': {
            'activation': 0.05,
            'fiber_length': 0.06027044615978444,
            'fiber_velocity': 2.2362024955832438e-15,
            'fiber_force': 99.63652479982161
        },
        'vasti_l': {
            'activation': 0.05,
            'fiber_length': 0.07890756873654925,
            'fiber_velocity': 6.168233828156989e-15,
            'fiber_force': 437.7385693769557
        },
        'gastroc_l': {
            'activation': 0.05,
            'fiber_length': 0.05720257668702345,
            'fiber_velocity': 5.718949639274886e-14,
            'fiber_force': 282.79456495087237
        },
        'soleus_l': {
            'activation': 0.05,
            'fiber_length': 0.04494814124106819,
            'fiber_velocity': 3.4120643802478774e-10,
            'fiber_force': 406.4161372187392
        },
        'tib_ant_l': {
            'activation': 0.05,
            'fiber_length': 0.06288000983990409,
            'fiber_velocity': 8.525642140971546e-14,
            'fiber_force': 104.5158690359632
        }
    },
    'markers': {},
    'misc': {
        'mass_center_pos': [-0.08466565561225976, 0.9952730567231536, -0.003576087446004414],
        'mass_center_vel': [0.0, 0.0, 0.0],
        'mass_center_acc': [4.4008799039045146e-14, 2.570832209970726, -1.0237645330147003e-15]
    }
}
```

In [20]:
# From https://github.com/stanfordnmbl/osim-rl/blob/master/osim/env/osim.py#L452
history = {}

def add_partial_obs_to_history(model_obs):
    if history['t'] == model_obs:
        return
    
    # Initialize history with all timesteps as the current obs.
    if not 't' in history:
        history['t-3'] = model_obs
        history['t-2'] = model_obs
        history['t-1'] = model_obs
        history['t'] = model_obs

    # Move history back one timestep and add latest obs.
    history['t-3'] = history['t-2']
    history['t-2'] = history['t-1']
    history['t-1'] = history['t']
    history['t'] = model_obs
    
def prepare_model_observation(env_obs):
    state_desc = env_obs
    prosthetic = 'pros_foot_r' in state_desc['body_pos']
    target_vel_x = 3
    target_vel_z = 0

    lower_order = []
    higher_order = []
    
    cog = {
        'pos': np.array(state_desc['misc']['mass_center_pos']),
        'vel': np.array(state_desc['misc']['mass_center_vel']),
        'acc': np.array(state_desc['misc']['mass_center_acc']),
    }
    for body_part in ["head","torso","pelvis","femur_l","femur_r","tibia_l","tibia_r","pros_tibia_r","talus_l","talus_r","toes_l","toes_r","pros_foot_r"]:
        if self.prosthetic and body_part in ["toes_r","tibia_r","talus_r"]:
            res += [0] * 9
            continue
        if not self.prosthetic and body_part in ["pros_foot_r","pros_tibia_r"]:
            res += [0] * 9
            continue
        lower_order += list(np.array(state_desc["body_pos"][body_part][0:2]) - cog['pos'])
        lower_order += list((np.array(state_desc["body_vel"][body_part][0:2]) - cog['vel']) / cog['vel'])
        higher_order += list((np.array(state_desc["body_acc"][body_part][0:2]) - cog['acc']) / cog['acc'])
        lower_order += state_desc["body_pos_rot"][body_part][0:2]
        lower_order += state_desc["body_vel_rot"][body_part][0:2]
        higher_order += state_desc["body_acc_rot"][body_part][0:2]

    for joint in ["ankle_l","ankle_r","back","hip_l","hip_r","knee_l","knee_r"]:
        lower_order += state_desc["joint_pos"][joint]
        lower_order += state_desc["joint_vel"][joint]
        higher_order += state_desc["joint_acc"][joint]

    for muscle in sorted(state_desc["muscles"].keys()):
        higher_order += [state_desc["muscles"][muscle]["activation"]]
        higher_order += [state_desc["muscles"][muscle]["fiber_length"]]
        higher_order += [state_desc["muscles"][muscle]["fiber_velocity"]]

    # TODO: difference between COG velocity and target velocity
    higher_order += [cog['vel'][0] - target_vel_x, cog['vel'][2] - target_vel_z]
    higher_order += cog['acc']
        
    add_partial_obs_to_history(np.array(higher_order))
    return np.concatenate(lower_order, history['t'], history['t-1'], history['t-2'], history['t-3'])



### Action hacking

- binary activations?  Or several "bins"?
- Frameskip?
- Muscles must remain "active" for at least 10 frames once activated?  Randomized?


In [ ]:
def prepare_env_action(model_action):
    
    # TODO 
    
    return model_action
    

### Episode Hacking (Custom "done" criteria)


In [5]:
def should_abort_episode(env):
    observation = env.get_state_desc()
#     print((np.array(observation['body_pos_rot']['torso'])*180/math.pi > 60).any())
#     if observation['body_pos_rot']['torso'][2] < -0.2:
#         return True
    if observation['body_pos']['head'][0] - observation['body_pos']['pelvis'][0] < -.1:
        print('Aborting episode due to head being > .1m behind the pelvis.')
        return True
    if np.fabs(observation['body_pos']['head'][2]) > 0.5:
        print('Aborting episode due to head being > 0.5m away from centerline.')
        return True
    rewards = compute_rewards(observation)
    if sum(rewards.values()) < -10:
        print('Aborting episode due to negative score (< -10).')
        return True
    return False
    

### Reward Hacking

- Survival reward
- Lean forward reward (to avod models which had torso too upright) (may need to be based on speed)
- Reward for minimizing sideways velocity?
- Reward for minimizing vertical velocity (COG)?

In [16]:
def compute_rewards(obs):
    if type(obs) != dict:
        raise ValueError('obs must be a dict (project=False)')

    target_vel_x = 3
    target_vel_z = 0
    eps = 1e-8

    target_vel_theta = -np.arctan(target_vel_z/(target_vel_x+eps)) if target_vel_x >= 0 else np.pi - np.arctan(target_vel_z/(target_vel_x+eps))

    # Parabolic reward/penalty for tracking a target value within some tolerance. 
    # Unit reward: 1 at target value, 0 at `tolerance` distance from target value, negative outside of `tolerance`.
    # Multiply by desired scale factor / magnitude. Don't multiply by too large of a scale factor — it also amplifies the slope.
    # Make the tolerance bigger than you think — it makes the slope more gradual / less severe.
    # `tolerance`: reward is positive if within `tolerance` of target value, else negative.
    val_diff_reward = lambda val, target, tolerance: (1 - ((val - target)/tolerance)**2)
    def radians_diff_wrapped(a1, a2):
        # Make both angles within (-2pi, 2pi)
        a1, a2 = a1 % (2*np.pi), a2 % (2*np.pi)
        # Make both angles positive -- within [0, 2pi)
        a1 = a1 + 2*np.pi if a1 < 0 else a1
        a2 = a2 + 2*np.pi if a2 < 0 else a2
        # Make a1 the smaller angle
        a1, a2 = min(a1, a2), max(a1, a2)
        # Make sure a1 is within pi of a2 — two angles can't be greater than pi apart in relative (wrapped) sense.
        a1 = a1 + 2*np.pi if a2 - a1 > np.pi else a1
        return np.fabs(a2 - a1)
    
    penalty += (state_desc["body_vel"]["pelvis"][0] - state_desc["target_vel"][0])**2
    return {
        'survival': 2,
        'target_velocity_x': 3 * val_diff_reward(obs['misc']['mass_center_vel'][0], target_vel_x, 5), # 3 at target velocity, 0 at 3m/s off-target, then negative
        'target_velocity_z': 3 * val_diff_reward(obs['misc']['mass_center_vel'][2], target_vel_z, 5), # 3 at target velocity, 0 at 3m/s off-target, then negative
        'head_velocity_x': 2 * val_diff_reward(obs['body_vel']['head'][0], target_vel_x, 5), # 2 at target velocity, 0 at 3m/s off-target, then negative
        'head_velocity_z': 2 * val_diff_reward(obs['body_vel']['head'][2], target_vel_z, 5), # 2 at target velocity, 0 at 3m/s off-target, then negative
        'lean_forward_x': 5 * val_diff_reward(obs['body_pos']['head'][0] - obs['body_pos']['pelvis'][0], .1 * target_vel_x, .4), # head in front of pelvis from perspective of velocity vector
        'lean_forward_z': 5 * val_diff_reward(obs['body_pos']['head'][2] - obs['body_pos']['pelvis'][2], .1 * target_vel_z, .4), # head in front of pelvis from perspective of velocity vector
        'hips_squared': 5 * val_diff_reward(radians_diff_wrapped(target_vel_theta, obs['body_pos_rot']['pelvis'][1]), 0, np.pi),
        'knee_bent_l': 5 * val_diff_reward(obs['joint_pos']['knee_l'][0], 60*np.pi/180, np.pi), # goal range of roughly [0,120] degrees
        'knee_bent_r': 5 * val_diff_reward(obs['joint_pos']['knee_r'][0], 60*np.pi/180, np.pi), # goal range of roughly [0,120] degrees
        'low_y_vel_knee_l': 5 * val_diff_reward(obs['body_vel']['knee_l'][1], 0, 1),
        'low_y_vel_knee_r': 5 * val_diff_reward(obs['body_vel']['knee_r'][1], 0, 1),
        'low_y_vel_toes_l': 5 * val_diff_reward(obs['body_vel']['toes_l'][1], 0, 1),
        'low_y_vel_pros_foot_r': 5 * val_diff_reward(obs['body_vel']['pros_foot_r'][1], 0, 1),
        'low_y_vel_pelvis': 5 * val_diff_reward(obs['body_vel']['pelvis'][1], 0, 1),
        'low_y_vel_head': 5 * val_diff_reward(obs['body_vel']['head'][1], 0, 1),
        'knees_opposite_joint_vel': 10 * val_diff_reward(obs['joint_vel']['knee_l'][0], -obs['joint_vel']['knee_r'][0], np.pi), # The left knee should be opening when the right knee is closing, and vice versa
        'feet_behind_mass_x': 5 * val_diff_reward(obs['misc']['mass_center_pos'][0] - (obs['body_pos']['toes_l'][0] + obs['body_pos']['pros_foot_r'][0])/2, .1 * target_vel_x, .4),
        'feet_behind_mass_z': 5 * val_diff_reward(obs['misc']['mass_center_pos'][2] - (obs['body_pos']['toes_l'][2] + obs['body_pos']['pros_foot_r'][2])/2, .1 * target_vel_z, .4),
        'one_foot_off_ground': 0,
        'femurs_parallel': 0,
        'absolute_foot_velocity': 0, # should be 0 at ground, 2x body velocity above ground
        'forefoot_strike': 0,
    }    
    

### Policy Evaluator

In [7]:
# Runs policy for X episodes and returns average reward
def run_episode(policy, custom_done_fn=None):
    obs = env.reset(**env_step_kwargs)
    obs = env.reset(**env_step_kwargs)
    done = False
    total_reward = 0
    while not done:
        action = policy.select_action(prepare_model_observation(obs))
        action = prepare_env_action(action)
        obs, reward, done, _ = env.step(action, **env_step_kwargs)
        
        # We don't use the custom rewards here because we want to evaluate our progress against the environment's reward.
        # obs_dict = env.get_state_desc()
        # custom_rewards = compute_rewards(obs_dict)
        # total_rewared += reward + sum(custom_rewards.values())

        total_reward += reward
        if not done and callable(custom_done_fn):
            done = custom_done_fn(env)
    return total_reward

def evaluate_policy(policy, eval_episodes=10, custom_done_fn=None):
    avg_reward = 0.
    for _ in range(eval_episodes):
        avg_reward += run_episode(policy, custom_done_fn=custom_done_fn)

    avg_reward /= eval_episodes

    print("---------------------------------------")
    print("Evaluation over %d episodes: %f" % (eval_episodes, avg_reward))
    print("---------------------------------------")
    return avg_reward



In [8]:
env = ProstheticsEnv(visualize=False, integrator_accuracy=CONFIG['env']['integrator_accuracy'])
env_step_kwargs = {'project': False}

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [9]:
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
max_action = int(env.action_space.high[0])


In [10]:
policy = TD3(state_dim, action_dim, max_action)

In [11]:
replay_buffer = ReplayBuffer()

In [12]:
%%timeit -n1 -r1
evaluations = [evaluate_policy(policy, custom_done_fn=should_abort_episode)]

NameError: name 'self' is not defined

In [11]:
total_timesteps = 0
timesteps_since_eval = 0
episode_num = 0
done = True 

In [12]:
while total_timesteps < CONFIG['training']['max_timesteps']:
    if done: 
        if total_timesteps != 0: 
            print(f"Total T: {total_timesteps} Episode Num: {episode_num} Episode T: {episode_timesteps} Reward: {episode_reward}")
            if CONFIG['model']['architecture'] == "TD3":
                policy.train(
                    replay_buffer, 
                    episode_timesteps, 
                    CONFIG['training']['batch_size'], 
                    CONFIG['training']['discount'], 
                    CONFIG['training']['tau'], 
                    CONFIG['training']['policy_noise'], 
                    CONFIG['training']['noise_clip'], 
                    CONFIG['training']['policy_freq']
                )
            else: 
                policy.train(replay_buffer, episode_timesteps, CONFIG['training']['batch_size'], CONFIG['training']['discount'], CONFIG['training']['tau'])
        
        # Evaluate episode
        if timesteps_since_eval >= CONFIG['training']['eval_freq']:
            timesteps_since_eval %= CONFIG['training']['eval_freq']
            evaluations.append(evaluate_policy(policy, custom_done_fn=should_abort_episode))
            torch.save(policy.state_dict(), CHECKPOINTS_DIR/file_name)
        
        # Reset environment
        obs = env.reset(**env_step_kwargs)
        obs_dict = env.get_state_desc()
        done = False
        episode_reward = 0
        episode_timesteps = 0
        episode_num += 1 
    
    # Select action randomly or according to policy
    if total_timesteps < CONFIG['training']['start_timesteps']:
        action = env.action_space.sample()
    else:
        action = policy.select_action(prepare_model_observation(obs))
        if CONFIG['training']['expl_noise'] != 0: 
            action = (action + np.random.normal(0, CONFIG['training']['expl_noise'], size=env.action_space.shape[0])).clip(env.action_space.low, env.action_space.high)

    # Perform action
    action = prepare_env_action(action)
    new_obs, reward, done, _ = env.step(action, **env_step_kwargs)
    new_obs_dict = env.get_state_desc()

    if not done:
        done = should_abort_episode(env)
    done_bool = 0 if episode_timesteps + 1 == CONFIG['training']['max_episode_steps'] else float(done)

    custom_rewards = compute_rewards(obs_dict)
    episode_reward += reward + sum(custom_rewards.values())

    # Store data in replay buffer
    replay_buffer.add((obs_dict, new_obs_dict, action, reward, done_bool, episode_num))

    obs = new_obs
    obs_dict = new_obs_dict

    episode_timesteps += 1
    total_timesteps += 1
    timesteps_since_eval += 1
    

Total T: 95 Episode Num: 1 Episode T: 95 Reward: -543.9680248189658
Total T: 192 Episode Num: 2 Episode T: 97 Reward: -559.8757033004392
Total T: 288 Episode Num: 3 Episode T: 96 Reward: -540.4079581627547
Total T: 386 Episode Num: 4 Episode T: 98 Reward: -563.3346533803907
Total T: 482 Episode Num: 5 Episode T: 96 Reward: -555.292968682621
Total T: 578 Episode Num: 6 Episode T: 96 Reward: -550.2183485309434
Total T: 674 Episode Num: 7 Episode T: 96 Reward: -541.1373096478648
Total T: 770 Episode Num: 8 Episode T: 96 Reward: -563.6875861703782
Total T: 867 Episode Num: 9 Episode T: 97 Reward: -568.2517473832543
Total T: 964 Episode Num: 10 Episode T: 97 Reward: -565.1752790861794
Total T: 1060 Episode Num: 11 Episode T: 96 Reward: -559.6357161388767
Total T: 1158 Episode Num: 12 Episode T: 98 Reward: -565.3874770336167
Total T: 1253 Episode Num: 13 Episode T: 95 Reward: -550.3142296671739
Total T: 1350 Episode Num: 14 Episode T: 97 Reward: -567.3470015031003
Total T: 1446 Episode Num: 

RuntimeError: std::exception in 'SimTK::State const & OpenSim::Manager::integrate(double)': SimTK Exception thrown at AbstractIntegratorRep.cpp:428:
  Integrator step failed at time 0.35336897974972997 apparently because:
SimTK Exception thrown at AbstractIntegratorRep.cpp:547:
  Error detected by Simbody method AbstractIntegrator::takeOneStep(): Unable to advance time past 0.353369.
  (Required condition 't1 > t0' was not met.)


In [51]:
!sudo shutdown

Shutdown scheduled for Wed 2018-08-29 06:23:47 UTC, use 'shutdown -c' to cancel.
